In [1]:
import os
import hddm
import pickle
from kabuki.analyze import gelman_rubin
import kabuki
from IPython.parallel import Client

/Users/minheeyoo/opt/anaconda3/envs/hddmenv/lib/python3.6/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


# Description of model and data 
* z = constrained to be 0.5
* v = v0 + v1 * tVD + v2 * cVD
    * tVD : target value difference (higher value target - lower value target)]
    * cVD: context value difference

In [2]:
data = hddm.load_csv('Model2_data.csv')
data

,subj_idx,response,rt,trialType,tVD,cVD,tVDcVD,sumVD
0,1,1,0.58297,hh,0.096667,0.000000,0.000000,0.096667
1,1,1,0.33312,ll,0.081000,-0.042222,-0.003420,0.038778
2,1,0,0.48313,Hh,0.086667,1.833333,0.158889,1.920000
3,1,1,0.86631,Hl,0.234333,-1.888889,-0.442630,-1.654556
4,1,1,0.54919,Hh,0.096667,1.832222,0.177115,1.928889
...,...,...,...,...,...,...,...,...
4085,24,0,0.57208,Hl,0.067333,-2.000000,-0.134667,-1.932667
4086,24,1,0.48706,ll,0.121000,0.000000,0.000000,0.121000
4087,24,1,0.58487,ll,0.193333,0.000000,0.000000,0.193333
4088,24,1,0.39946,hh,0.045667,0.073333,0.003349,0.119000


In [3]:
def run_model(i):
    import os
    import hddm
    import pickle
    pathAnalysis = 'project'
    os.chdir(pathAnalysis)
    data = hddm.load_csv(pathAnalysis + '/' + 'Model2_data.csv')
    # Fit model
    v_reg = {'model': 'v ~ 1 + tVD + cVD', 
             'link_func': lambda x: x}
    reg_descr = [v_reg]
    m = hddm.HDDMRegressor(data, reg_descr, group_only_regressors=False)
    m.sample(10000, burn=4000, dbname='Model2d_chain%i.db'%i, db='pickle')
    return m

In [4]:
v = Client()[:]
jobs = v.map(run_model, range(3)) # 4 is the number of CPUs
model_set = jobs.get()

In [5]:
# save model set
with open('Model2d_hddm_model_set', 'wb') as f:
    pickle.dump(model_set, f)

In [6]:
models = kabuki.utils.concat_models(model_set)

In [7]:
gelman_rubin(model_set)

{'a': 0.9999449978361485,
 'a_std': 1.0003737008798865,
 'a_subj.1': 1.0000069978645247,
 'a_subj.2': 0.9999702894913511,
 'a_subj.3': 0.9999557687225512,
 'a_subj.4': 1.0003245933016491,
 'a_subj.5': 1.000121737485326,
 'a_subj.6': 1.0000671972194288,
 'a_subj.7': 1.0000508412188067,
 'a_subj.8': 1.0007174521250481,
 'a_subj.9': 1.0001080631847596,
 'a_subj.10': 1.0000585306963785,
 'a_subj.11': 1.0002930991446015,
 'a_subj.12': 1.0001134309228383,
 'a_subj.13': 1.000318414664341,
 'a_subj.14': 0.9999481764500696,
 'a_subj.15': 1.0008381701916833,
 'a_subj.16': 0.999975376126973,
 'a_subj.17': 1.000288105029,
 'a_subj.18': 1.000138731292942,
 'a_subj.19': 0.9999414935612198,
 'a_subj.20': 1.0001391939679687,
 'a_subj.21': 0.9999184184632356,
 'a_subj.22': 1.0001475362554761,
 'a_subj.23': 0.9999568731943826,
 'a_subj.24': 1.0000947724429121,
 't': 1.0001707738859749,
 't_std': 1.0001435082882058,
 't_subj.1': 1.000286722410016,
 't_subj.2': 1.0000378465639885,
 't_subj.3': 0.999964824

In [10]:
import csv
tmp = models.dic_info
with open('Model2d_dic_info.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
    w = csv.DictWriter(f, tmp.keys())
    w.writeheader()
    w.writerow(tmp)